In [1]:
# Importing libraries
import os
from dotenv import load_dotenv
import requests
import gradio as gr
from openai import OpenAI
from bs4 import BeautifulSoup
from IPython.display import display, Markdown

In [3]:
# Load environment variables from the .env file
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [5]:
openai = OpenAI()

In [6]:
# Website class to handle the extraction of title and text content from a webpage
class Website():
    url: str
    title: str
    text: str
    
    def __init__(self,url):
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content,'html.parser')
        self.title = soup.title.string if soup.title else 'no title exist'
        for irrelevant in soup.body(['script', 'input','img','style']):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [7]:
# System prompt defines LLM task and toner --> summarizing website content
system_prompt = """
You are an assistant that analyzes website content and provides summaries. \
Respond in Markdown format and offer styles such as key points, detailed summaries or Q&A format.
"""

In [8]:
# Function to generate user-specific prompt based on style
def user_prompt(website, style):
    if style == "Key Points":
        user_prompt = f"Summarize the website titled '{website.title}' in key points:\n\n{website.text}"
    elif style == "Detailed":
        user_prompt = f"Provide a detailed summary of the website titled '{website.title}':\n\n{website.text}"
    elif style == "Q&A":
        user_prompt = f"Generate a question-answer format summary for the website titled '{website.title}':\n\n{website.text}"
    return user_prompt

In [9]:
# Define the function to structure the message for GPT
def message_for(website, style):
       return[{"role": "system", "content": system_prompt},
              {"role": "user","content": user_prompt(website, style)}]

In [10]:
# Define the summarize function to generate summary for given URL
def summarize(url, style):
    website = Website(url)
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages= message_for(website, style)
    )
    return response.choices[0].message.content

In [11]:
# Function to display this nicely in the Jupyter output
def display_summary(url, style):
    summary = summarize(url, style)
    display(Markdown(summary))

In [14]:
display_summary('https://www.nytimes.com/','Key Points')

# Summary of 'The New York Times - Breaking News, US News, World News and Videos'

### Key Points

- **Sections and Coverage**:
  - **U.S. News**: Politics, health, obituaries, climate, sports, and more.
  - **World News**: International stories focusing on regions like Africa, Asia, Middle East, and Europe.
  - **Business**: Coverage of economy, finances, tech, and market news.
  - **Arts and Lifestyle**: Includes today's arts, books, food, travel, style, and wellness.

- **Podcasts and Audio**:
  - Offers various podcasts such as *The Daily*, *The Run-Up*, *Matter of Opinion*, and more, focusing on deep dives into current events and topics.
  - Also provides narrated articles and audio segments.

- **Newsletters**:
  - Curated options like *The Morning* for daily news, *DealBook* for business updates, and *Open Thread* for fashion.

- **Opinion Sections**:
  - Includes guest essays, editorials, and columns from renowned writers, addressing a variety of topics including politics, culture, and health.

- **Interactive Features**:
  - Offers games like crossword puzzles, word games, and brain teasers.
  - Provides recommendations for books, TV shows, and movies through the Arts sections.

- **Current Events**:
  - Features ongoing major stories such as the Middle East crisis, the Russia-Ukraine war, and U.S. political updates.
  - Recent key articles focus on immigration, health issues, labor rights in Saudi Arabia, and environmental reports.

- **Lifestyle Content**:
  - Includes lifestyle guidance on health and wellness, cooking tips, travel advice, and personal finance.
  - Review and recommendation features through Wirecutter.

- **Engagement and Community**:
  - Encourages user interaction through comments, forums, and contributing tips or stories.

This comprehensive platform aims to inform and engage readers with a variety of news articles, interactive content, and podcasts tailored to various interests and regions.

In [15]:
# Gradio Interface
def main_interface(url, style):
    if url:
        summary = summarize(url, style)
        return summary
    else:
        return "Please provide a URL."

# Gradio app
interface = gr.Interface(
    fn=main_interface,
    inputs=[
        gr.Textbox(label="Enter URL for Summarization"),
        gr.Radio(["Key Points", "Detailed", "Q&A"], label="Choose Summary Style", value="Key Points"),
    ],
    outputs=[
        gr.Markdown(label="Summary")
    ],
    title="Website Summarizer",
    description="Paste a URL and choose a summary style."
)

# Launch the Gradio app
interface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
